In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#!mkdir -p drive/My\ Drive/Colab\ Notebooks/data/.kaggle
#!cp kaggle.json drive/My\ Drive/Colab\ Notebooks/data/.kaggle/
# Permission Warning 이 일어나지 않도록 
#!chmod 600 drive/My\ Drive/Colab\ Notebooks/data/.kaggle/kaggle.json
# 본인이 참가한 모든 대회 보기 
#!kaggle competitions list

In [0]:
!pwd

/content


In [0]:
#!cd drive/My\ Drive/Colab\ Notebooks/data; mkdir distracted-driver-detection

In [0]:
#!cd drive/My\ Drive/Colab\ Notebooks/data/distracted-driver-detection; kaggle competitions download -c state-farm-distracted-driver-detection

In [0]:
#!cd drive/My\ Drive/Colab\ Notebooks/data/distracted-driver-detection;unzip state-farm-distracted-driver-detection

In [0]:
!cd drive/My\ Drive/Colab\ Notebooks/data/distracted-driver-detection; ls -l

total 8
drwx------ 2 root root 4096 Feb 13 14:06 imgs
-rw------- 1 root root    0 Feb 15 18:13 my_checkpoint.data-00000-of-00002
-rw------- 1 root root    0 Feb 15 18:13 my_checkpoint.data-00001-of-00002
drwx------ 2 root root 4096 Feb 15 18:13 my_checkpoint_temp_ab66df4e71234964a58b92daa037790a


In [0]:
%tensorflow_version 2.x

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

TensorFlow 2.x selected.


In [0]:
sample_img=cv2.imread("drive/My Drive/Colab Notebooks/data/distracted-driver-detection/imgs/train/c0/img_34.jpg",cv2.IMREAD_COLOR)

In [0]:
sample_img.shape

(480, 640, 3)

In [0]:
input_size=sample_img.shape

In [0]:
import random

train_X=np.empty((0,input_size[0],input_size[1],3))
train_Y=np.empty((0,1))

max_count=300

for i in range(10):
  dirname='drive/My Drive/Colab Notebooks/data/distracted-driver-detection/imgs/train/c'+str(i)+'/'
  filenames = os.listdir(dirname)
  random.shuffle(filenames)
  count=0
  for filename in filenames:
    
    full_filename = os.path.join(dirname, filename)
    image_array=cv2.imread(full_filename,cv2.IMREAD_COLOR)
    #image_array = cv2.resize(image_array, dsize=(input_size[1], input_size[0]), interpolation=cv2.INTER_LINEAR)

    train_X=np.append(train_X,np.array([image_array]),axis=0)
    train_Y=np.append(train_Y,np.array([[i]]),axis=0)
    count+=1

    if count>=max_count:
      break;

  print("c"+str(i)+" all loaded!")

c0 all loaded!


In [0]:
print(train_X.shape)
print(train_Y.shape)

In [0]:
mask=np.arange(train_Y.shape[0])
np.random.shuffle(mask)

In [0]:
train_Y=train_Y[mask]
train_X=train_X[mask]

In [0]:
np.save('drive/My Drive/Colab Notebooks/data/distracted-driver-detection/train_X',train_X)
np.save('drive/My Drive/Colab Notebooks/data/distracted-driver-detection/train_Y',train_Y)
!cd drive/My\ Drive/Colab\ Notebooks/data/distracted-driver-detection; ls -l

In [0]:
!cd drive/My\ Drive/Colab\ Notebooks/data/distracted-driver-detection; ls -l

In [0]:
test_X=np.empty((0,input_size[0],input_size[1],3))
test_id=np.empty((0,1))


testdir='drive/My Drive/Colab Notebooks/data/distracted-driver-detection/imgs/test/'
filenames=os.listdir(testdir)
random.shuffle(filenames)

print(len(filenames))
count=0

for filename in filenames:
  full_path=os.path.join(testdir,filename)
  image_array=cv2.imread(full_filename,cv2.IMREAD_COLOR)
  #image_array = cv2.resize(image_array, dsize=(input_size[1], input_size[0]), interpolation=cv2.INTER_LINEAR)

  test_X=np.append(test_X,np.array([image_array]),axis=0)
  test_id=np.append(test_id,np.array([[filename]]),axis=0)

  count+=1
  if count>=200:
      break;

print(test_X.shape)
print(test_id.shape)

In [0]:
np.save('drive/My Drive/Colab Notebooks/data/distracted-driver-detection/test_X',test_X)
np.save('drive/My Drive/Colab Notebooks/data/distracted-driver-detection/test_id',test_id)

In [0]:
!cd drive/My\ Drive/Colab\ Notebooks/data/distracted-driver-detection; ls -l

In [0]:
# vgg 16
model=tf.keras.Sequential([
                           tf.keras.layers.Conv2D(input_shape=(input_shape[0],input_shape[1],3),kernel_size=(3,3),filters=64,padding='same',activation='relu'),
                           tf.keras.layers.Conv2D(kernel_size=(3,3),filters=64,padding='same',activation='relu'),
                           tf.keras.layers.MaxPool2D(pool_size=(2,2)), 
                           tf.keras.layers.Dropout(rate=0.5),

                           tf.keras.layers.Conv2D(kernel_size=(3,3),filters=128,padding='same',activation='relu'),
                           tf.keras.layers.Conv2D(kernel_size=(3,3),filters=128,padding='same',activation='relu'),
                           tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                           tf.keras.layers.Dropout(rate=0.5),

                           tf.keras.layers.Conv2D(kernel_size=(3,3),filters=256,padding='same',activation='relu'),
                           tf.keras.layers.Conv2D(kernel_size=(3,3),filters=256,padding='same',activation='relu'),
                           tf.keras.layers.Conv2D(kernel_size=(3,3),filters=256,padding='same',activation='relu'),
                           tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                           tf.keras.layers.Dropout(rate=0.5),

                           tf.keras.layers.Conv2D(kernel_size=(3,3),filters=512,padding='same',activation='relu'),
                           tf.keras.layers.Conv2D(kernel_size=(3,3),filters=512,padding='same',activation='relu'),
                           tf.keras.layers.Conv2D(kernel_size=(3,3),filters=512,padding='same',activation='relu'),
                           tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                           tf.keras.layers.Dropout(rate=0.5),

                           tf.keras.layers.Conv2D(kernel_size=(3,3),filters=512,padding='same',activation='relu'),
                           tf.keras.layers.Conv2D(kernel_size=(3,3),filters=512,padding='same',activation='relu'),
                           tf.keras.layers.Conv2D(kernel_size=(3,3),filters=512,padding='same',activation='relu'),
                           tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                           tf.keras.layers.Dropout(rate=0.5),

                           tf.keras.layers.Flatten(),
                           tf.keras.layers.Dense(units=4096,activation='relu'),
                           tf.keras.layers.Dropout(rate=0.5),
                           tf.keras.layers.Dense(units=4096,activation='relu'),
                           tf.keras.layers.Dropout(rate=0.5),
                           tf.keras.layers.Dense(units=10,activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy'],)

model.summary()

In [0]:
history=model.fit(train_X,train_Y,epochs=300,validation_split=0.2)

In [0]:
plt.figure(figsize=(12,4))

plt.subplot(1,2,1)
plt.plot(history.history['loss'],'b-',label='loss')
plt.plot(history.history['val_loss'],'r--',label='val_loss')
plt.ylim(0,1)
plt.xlabel('epoch')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['accuracy'],'g-',label='accuracy')
plt.plot(history.history['val_accuracy'],'k--',label='val_accuracy')
plt.ylim(0,1)
plt.xlabel('epoch')
plt.legend()

plt.show()

In [0]:
plt.figure(figsize=(20,10))
for i in range (5):
  plt.subplot(1,5,i+1)
  img=cv2.cvtColor(test_X[i].reshape(224,224,3),cv2.COLOR_BGR2RGB)
  plt.imshow(img)
for i in range (5):
  plt.subplot(2,5,i+1)
  img=cv2.cvtColor(test_X[5+i].reshape(224,224,3),cv2.COLOR_BGR2RGB)
  plt.imshow(img)

plt.show()

print(np.argmax(model.predict(test_X[0:10]),axis=1))

In [0]:
model.save_weights('drive/My Drive/Colab Notebooks/data/distracted-driver-detection/my_checkpoint')